In [31]:
import pandas as pd
import numpy as np

train_df = pd.read_csv('new1_cyber_train.csv')
test_df = pd.read_csv('new1_cyber_test.csv')

In [34]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from spellchecker import SpellChecker
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import re

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Remove special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    # Apply stemming
    ps = PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

train_df['text'] = train_df['text'].apply(preprocess_text)
test_df['text'] = test_df['text'].apply(preprocess_text)

# Tokenization for DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', do_lower_case=True)

# Encoding as numbers for training dataset
encoded_train_texts = tokenizer(train_df['text'].tolist(), padding=True, truncation=True, return_tensors='tf')
X_train_padded = pad_sequences(encoded_train_texts['input_ids'], padding='post')
y_train = train_df['label'].to_numpy()



# Encoding as numbers for testing dataset
encoded_test_texts = tokenizer(test_df['text'].tolist(), padding=True, truncation=True, return_tensors='tf')
X_test_padded = pad_sequences(encoded_test_texts['input_ids'], padding='post')
y_test = test_df['label'].to_numpy()



# Building the DistilBERT model
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Compile the model (define optimizer, loss, and metric)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Training the model
history = model.fit(
    {'input_ids': X_train_padded, 'attention_mask': encoded_train_texts['attention_mask']},
    y_train,
    epochs=3,
    batch_size=32,
    validation_split=0.1
)

# Evaluating the model on the testing dataset
eval_result = model.evaluate(
    {'input_ids': X_test_padded, 'attention_mask': encoded_test_texts['attention_mask']},
    y_test
)
print(f"Test accuracy: {eval_result[1]}")


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/3

26/26 [==============================] - 369s 13s/step - loss: 0.6854 - accuracy: 0.5425 - val_loss: 0.6676 - val_accuracy: 0.6813
Epoch 2/3
26/26 [==============================] - 338s 13s/step - loss: 0.6245 - accuracy: 0.7300 - val_loss: 0.5600 - val_accuracy: 0.7912
Epoch 3/3
8/8 [==============================] - 9s 788ms/step - loss: 0.3780 - accuracy: 0.8496
Test accuracy: 0.8495575189590454
